##INSAID Capstone project - group 1015

###Installing and importing mysql.connector/python

In [358]:
!pip install --user mysql-connector-python

In [0]:
import mysql.connector as sql
import pandas as pd

###Creating db connection and storing in capdb

In [0]:
capdb = sql.connect(host='cpanel.insaid.co', database='Capstone1', user='student', password='student')

###Pulling data from SQL to Pandas df

In [361]:
print(pd.read_sql("show tables",con=capdb))

        Tables_in_Capstone1
0               events_data
1          gender_age_train
2  phone_brand_device_model


In [362]:
print(pd.read_sql("describe events_data",con=capdb))

       Field         Type Null Key Default Extra
0   event_id   bigint(20)   NO        None      
1  device_id  bigint(255)  YES        None      
2  timestamp     datetime   NO        None      
3  longitude   float(8,6)  YES        None      
4   latitude   float(8,6)  YES        None      
5       city  varchar(50)   NO        None      
6      state  varchar(50)   NO        None      


In [363]:
print(pd.read_sql("describe gender_age_train",con=capdb))

       Field           Type Null Key Default Extra
0  device_id    bigint(255)   NO        None      
1     gender  varchar(2555)   NO        None      
2        age       int(255)   NO        None      
3      group  varchar(2555)   NO        None      


In [364]:
print(pd.read_sql("describe phone_brand_device_model",con=capdb))

          Field           Type Null Key Default Extra
0     device_id    bigint(255)   NO        None      
1   phone_brand  varchar(2555)   NO        None      
2  device_model  varchar(2555)   NO        None      


In [0]:
query = """select *
          from events_data
          where state = 'delhi'"""
events_data_delhi = pd.read_sql(query, con=capdb)
events_data_delhi.shape

In [366]:
query = """select *
          from gender_age_train"""
gender_age_train = pd.read_sql(query, con=capdb)
gender_age_train.shape

(74645, 4)

In [367]:
query = """select *
          from phone_brand_device_model"""
phone_brand_device_model = pd.read_sql(query, con=capdb)
phone_brand_device_model.shape

(87726, 3)

### Closing db connection

In [0]:
capdb.close()